## Final Project
### Xi Yang 16216573

### data
** the data set are from i2b2 which needs auth to public, so I could not shared the data, but example data are described below **
- training data: C:\Users\xiyang\Google Drive\NLP_Project\i2b2_data\concept_assertion_relation_training_data
    1. concept: beth/concept/*.con; partners/concept/*.con
    2. txt: beth/txt/*.txt; partners/txt/*.txt
- testing data: C:\Users\xiyang\Google Drive\NLP_Project\i2b2_data\reference_standard_for_test_data
    1. concept: concepts/*.con
    2. txt: test_data/*.txt

### data sample
** .con and .txt files are linked by sharing same filename **
- data in .con file: c="angap" 65:25 65:25||t="test"
- data in .txt file: 2018-10-31 06:25 AM BLOOD Glucose - 91 UreaN - 19 Creat - 0.8 Na - 134 K - 4.0 Cl - 98 HCO3 - 26 _AnGap_ - 14

In [2]:
#read in training and testing data as raw
import glob
from functools import reduce

train_base_path = "C:\\Users\\xiyang\\Google Drive\\NLP_Project\\i2b2_data\\concept_assertion_relation_training_data"

#print(list(map(lambda x: glob.glob(x),  [files for files in con_file_paths])))

train_con_file_paths = [train_base_path + "\\beth\\concept\\*.con", train_base_path + "\\partners\\concept\\*.con"]
train_txt_file_paths = [train_base_path + "\\beth\\txt\\*.txt", train_base_path + "\\partners\\txt\\*.txt"]

train_con_files = list(reduce(lambda y, z: y + z, list(map(lambda x: glob.glob(x),  [files for files in train_con_file_paths]))))
train_txt_files = list(reduce(lambda y, z: y + z, list(map(lambda x: glob.glob(x),  [files for files in train_txt_file_paths]))))

#print(len(con_files))
#print(len(txt_files))

#in each file pair, the first is the txt and second is the associated concept file
train_file_pairs = list(zip(train_txt_files, train_con_files))
print(len(train_file_pairs))


test_base_path = "C:\\Users\\xiyang\\Google Drive\\NLP_Project\\i2b2_data\\reference_standard_for_test_data"

#print(list(map(lambda x: glob.glob(x),  [files for files in con_file_paths])))

test_con_file_paths = [test_base_path + "\\concepts\\*.con"]
test_txt_file_paths = [test_base_path + "\\test_data\\*.txt"]

test_con_files = list(reduce(lambda y, z: y + z, list(map(lambda x: glob.glob(x),  [files for files in test_con_file_paths]))))
test_txt_files = list(reduce(lambda y, z: y + z, list(map(lambda x: glob.glob(x),  [files for files in test_txt_file_paths]))))

#print(len(con_files))
#print(len(txt_files))

#in each file pair, the first is the txt and second is the associated concept file
test_file_pairs = list(zip(test_txt_files,test_con_files))
print(len(test_file_pairs))

170
256


In [75]:
'''
In txt file, each line is a sentence token and each word need to be tokenized
tokenize line into words listed in each line and pedding related tags based on information from con file
using BIO tagging
Example:
    concept: c="angap" 65:25 65:25||t="test"
    sentence: 98 HCO3 - 26 AnGap - 14
    output:
    98    o
    HCO3    o
    -    o
    26    o
    ANGap    B-test
    -    o
    14   o
'''
import re
import collections

def preprocess_raw_con_txt_file(file_pair):
    con_dict = dict()
    con_tag_dict = dict()
    txt = file_pair[0]
    con = file_pair[1]
    
    with open(con, "r") as fr:
        for i, line in enumerate(fr):
            info1 = line.split("||")
            #info2 = info1[0].split(" ")
            #print(re.findall(r'\"(.+?)\"', info1[0])[0])
            info2 = re.findall(r'[0-9]+:[0-9]+', info1[0])
            #print(info2)
            #get concept
            c = re.findall(r'\"(.+?)\"', info1[0])[0]
            
            #get tag label
            tl = re.findall(r'\"(.+?)\"', info1[1])[0]
            
            #tag B and I
            first_token_num = int(info2[0].split(":")[1])
            last_token_num = int(info2[1].split(":")[1])
            span = last_token_num - first_token_num
            
            line_num = int(info2[0].split(":")[0])
            
            if line_num not in con_dict:
                B_tags = dict()
                I_tags = dict()
                B_tags[first_token_num] = tl
                for i in range(span):
                    I_tags[first_token_num+i+1] = tl
                con_dict[line_num] = dict()
                con_dict[line_num]['B_tags'] = B_tags
                con_dict[line_num]['I_tags'] = I_tags
            else:
                con_dict[line_num]['B_tags'][first_token_num] = tl
                for i in range(span):
                     con_dict[line_num]['I_tags'][first_token_num+i+1] = tl
    #print(con_dict)
            
    with open(txt, "r") as fr:
        for line_index, line in enumerate(fr):
            tokens = line[:-1].split(" ")
            con_tag_dict[line_index+1] = []
            if line_index+1 in con_dict:
                ref_B_tags = con_dict[line_index+1]['B_tags']
                ref_I_tags = con_dict[line_index+1]['I_tags']
                #print(ref_B_tags)
                for token_index, token in enumerate(tokens):
                    if token_index in ref_B_tags:
                        new_tag = "-".join(["B", ref_B_tags[token_index]])
                    elif token_index in ref_I_tags:
                        new_tag = "-".join(["I", ref_I_tags[token_index]])
                    else:
                        new_tag = "O"
                    new_token = " ".join([token, new_tag])
                    #print(new_token)
                    con_tag_dict[line_index+1].append(new_token)
            else:
                for token in tokens:
                    new_token = " ".join([token, "O"])
                    con_tag_dict[line_index+1].append(new_token)
    
    return collections.OrderedDict(sorted(con_tag_dict.items()))
    
#test function
# p = ('C:\\Users\\xiyang\\Google Drive\\NLP_Project\\i2b2_data\\concept_assertion_relation_training_data\\beth\\txt\\record-105.txt', 'C:\\Users\\xiyang\\Google Drive\\NLP_Project\\i2b2_data\\concept_assertion_relation_training_data\\beth\\concept\\record-105.con')
# print(preprocess_raw_con_txt_file(p))

In [76]:
'''
create file with cornell-2000 format for training and testing
'''
train_file_name = "train.txt"
test_file_name = "test.txt"

def create_input(file_pairs, file_name):
    with open(file_name, "w") as f:
        for pair in file_pairs:
            tags = preprocess_raw_con_txt_file(pair)
            for k, v in tags.items():
                #print(k)
                for each in v:
                    print(each, file=f, end='\n')
                print('', file=f, end='\n')
            
create_input(train_file_pairs, train_file_name)
create_input(test_file_pairs, test_file_name)

In [4]:
#get train data word freq and word cloud
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

for each in train_file_pairs:
    with open(each[0], "r") as fr:
        for line in fr:
            print(line)

ImportError: No module named 'wordcloud'